# **Laboratorul 10 PMP**

In [12]:
import numpy as np
import pymc as pm
import arviz as az

publicity = np.array([1.5, 2.0, 2.3, 3.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6.0, 6.5, 7.0, 7.5, 8.0, 8.5, 9.0, 9.5, 10.0, 10.5, 11.0])

sales = np.array([5.2, 6.8, 7.5, 8.0, 9.0, 10.2, 11.5, 12.0, 13.5, 14.0, 15.0, 15.5, 16.2, 17.0, 18.0, 18.5, 19.5, 20.0, 21.0, 22.0])

new_publicity = np.array([2.0, 6.0, 10.0, 12.0])

print("Publicity:", publicity)
print("Sales:", sales)

with pm.Model() as model:
    alpha = pm.Normal("alpha", mu=0.0, sigma=10.0)
    beta  = pm.Normal("beta",  mu=0.0, sigma=5.0)
    sigma = pm.HalfCauchy("sigma", beta=5.0)

    mu = alpha + beta * publicity

    pm.Normal("sales", mu=mu, sigma=sigma, observed=sales)

    idata = pm.sample(draws=2000, tune=1000, target_accept=0.9, random_seed=2025, return_inferencedata=True)

print("\nPosterior summary (94% HDI) pentru coeficienti:")
summary = az.summary(idata, var_names=["alpha", "beta", "sigma"], hdi_prob=0.94)
print(summary)

with model:
    sales_new = pm.Normal(
        "sales_new",
        mu=alpha + beta * new_publicity,
        sigma=sigma,
    )

    ppc = pm.sample_posterior_predictive(
        idata,
        var_names=["sales_new"],
        random_seed=2025,
    )

y_pred = ppc.posterior_predictive["sales_new"].values
pred_int = np.percentile(y_pred, [5, 95], axis=(0, 1))

print("\nPredictive intervals (90%) pentru noi niveluri de publicitate:")
for x_new, lo, hi in zip(new_publicity, pred_int[0], pred_int[1]):
    print(f"publicity = {x_new:4.1f}k  ->  sales ~ [{lo:5.2f}k, {hi:5.2f}k]")


Publicity: [ 1.5  2.   2.3  3.   3.5  4.   4.5  5.   5.5  6.   6.5  7.   7.5  8.
  8.5  9.   9.5 10.  10.5 11. ]
Sales: [ 5.2  6.8  7.5  8.   9.  10.2 11.5 12.  13.5 14.  15.  15.5 16.2 17.
 18.  18.5 19.5 20.  21.  22. ]


Output()

Output()


Predictive intervals (90%) pentru noi niveluri de publicitate:
publicity =  2.0k  ->  sales ~ [ 6.06k,  7.51k]
publicity =  6.0k  ->  sales ~ [12.90k, 14.30k]
publicity = 10.0k  ->  sales ~ [19.73k, 21.15k]
publicity = 12.0k  ->  sales ~ [23.09k, 24.62k]
